In [1]:
import numpy as np
import pandas as pd
import scipy.linalg

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

import sys
print(sys.version)
sys.path.append('../')
import os
import copy

from models.powertrain.bounded_powertrain import Bounded_powertrain
from models.kinematic.ideal_diff_drive import Ideal_diff_drive
from models.learning.gp_slip import *
from models.kinematic.ICR_based import *
from models.kinematic.Perturbed_unicycle import *
from models.kinematic.enhanced_kinematic import *


from util.transform_algebra import *
from util.util_func import *

from sklearn import linear_model

3.10.6 (main, Mar 10 2023, 10:55:28) [GCC 11.3.0]


In [2]:
# import slip dataset

# dataset_path = '../data/ral2023_dataset/husky/boreal_snow/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/husky/grand_salon_tile_inflated/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/husky/grand_salon_left-deflated/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/boreal_snow/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/ga_hard_snow_a/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/ga_hard_snow_b/slip_dataset_all.pkl'
dataset_path = '../data/ral2023_dataset/marmotte/grand_salon_tile_b/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/warthog_tracks/boreal_mud/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/warthog_tracks/grand-axe_crusted-snow/slip_dataset_all.pkl'


full_dataset = pd.read_pickle(dataset_path)
full_dataset_length = len(full_dataset)
train_dataset = full_dataset[:int(full_dataset_length/2)]
validate_dataset = full_dataset[int(full_dataset_length/2):]

print(train_dataset)

     init_icp_x  init_icp_y  init_icp_z  init_icp_roll  init_icp_pitch  \
0           0.0         0.0         0.0            0.0             0.0   
1           0.0         0.0         0.0            0.0             0.0   
2           0.0         0.0         0.0            0.0             0.0   
3           0.0         0.0         0.0            0.0             0.0   
4           0.0         0.0         0.0            0.0             0.0   
..          ...         ...         ...            ...             ...   
311         0.0         0.0         0.0            0.0             0.0   
312         0.0         0.0         0.0            0.0             0.0   
313         0.0         0.0         0.0            0.0             0.0   
314         0.0         0.0         0.0            0.0             0.0   
315         0.0         0.0         0.0            0.0             0.0   

     init_icp_yaw  calib_step  cmd_left_0  cmd_right_0  cmd_left_1  ...  \
0             0.0         0.0   10.0

In [3]:
# isolate steady-state data

# extract cmd_body_vel arrays (input arrays)

idd_body_vel_x_str_list = []
idd_body_vel_y_str_list = []
idd_body_vel_yaw_str_list = []
for i in range(0, 40):
    str_idd_vel_x_i = 'idd_vel_x_' + str(i)
    str_idd_vel_y_i = 'idd_vel_y_' + str(i)
    str_idd_vel_yaw_i = 'idd_vel_yaw_' + str(i)
    idd_body_vel_x_str_list.append(str_idd_vel_x_i)
    idd_body_vel_y_str_list.append(str_idd_vel_y_i)
    idd_body_vel_yaw_str_list.append(str_idd_vel_yaw_i)
idd_body_vel_x_array = train_dataset[idd_body_vel_x_str_list].to_numpy()
idd_body_vel_y_array = train_dataset[idd_body_vel_y_str_list].to_numpy()
idd_body_vel_yaw_array = train_dataset[idd_body_vel_yaw_str_list].to_numpy()

x_train = np.column_stack((idd_body_vel_x_array.flatten(), idd_body_vel_yaw_array.flatten()))

# extract icp vels

str_icp_vel_x_list = []
str_icp_vel_y_list = []
str_icp_vel_yaw_list = []
for i in range(0, 40):
    str_icp_vel_x_i = 'icp_vel_x_' + str(i)
    str_icp_vel_y_i = 'icp_vel_y_' + str(i)
    str_icp_vel_yaw_i = 'icp_vel_yaw_' + str(i)
    str_icp_vel_x_list.append(str_icp_vel_x_i)
    str_icp_vel_y_list.append(str_icp_vel_y_i)
    str_icp_vel_yaw_list.append(str_icp_vel_yaw_i)
icp_vel_x_array = train_dataset[str_icp_vel_x_list].to_numpy()
icp_vel_y_array = train_dataset[str_icp_vel_y_list].to_numpy()
icp_vel_yaw_array = train_dataset[str_icp_vel_yaw_list].to_numpy()

# extract body_vel_distruptions arrays (output arrays)

str_body_vel_disturption_x_list = []
str_body_vel_disturption_y_list = []
str_body_vel_disturption_yaw_list = []
for i in range(0, 40):
    str_body_vel_disturption_x_i = 'body_vel_disturption_x_' + str(i)
    str_body_vel_disturption_y_i = 'body_vel_disturption_y_' + str(i)
    str_body_vel_disturption_yaw_i = 'body_vel_disturption_yaw_' + str(i)
    str_body_vel_disturption_x_list.append(str_body_vel_disturption_x_i)
    str_body_vel_disturption_y_list.append(str_body_vel_disturption_y_i)
    str_body_vel_disturption_yaw_list.append(str_body_vel_disturption_yaw_i)
    
body_vel_disturption_x_array = train_dataset[str_body_vel_disturption_x_list].to_numpy()
body_vel_disturption_y_array = train_dataset[str_body_vel_disturption_y_list].to_numpy()
body_vel_disturption_yaw_array = train_dataset[str_body_vel_disturption_yaw_list].to_numpy()

icp_x_str_list = []
icp_y_str_list = []
icp_z_str_list = []
icp_roll_str_list = []
icp_pitch_str_list = []
icp_yaw_str_list = []
for i in range(0, 40):
    str_icp_x_i = 'icp_interpolated_x_' + str(i)
    str_icp_y_i = 'icp_interpolated_y_' + str(i)
    str_icp_yaw_i = 'icp_interpolated_yaw_' + str(i)
    icp_x_str_list.append(str_icp_x_i)
    icp_y_str_list.append(str_icp_y_i)
    icp_yaw_str_list.append(str_icp_yaw_i)
# icp_x_array = validate_dataset[icp_x_str_list].to_numpy()
# icp_y_array = validate_dataset[icp_y_str_list].to_numpy()
# icp_yaw_array = validate_dataset[icp_yaw_str_list].to_numpy()

y_train_longitudinal_slip = body_vel_disturption_x_array.flatten()
y_train_lateral_slip = body_vel_disturption_y_array.flatten()
y_train_angular_slip = body_vel_disturption_yaw_array.flatten()

# compute mean body vel disturbance for each steady-state window
n_windows = len(train_dataset)

steady_state_mask = train_dataset['steady_state_mask'].to_numpy() == True

steady_state_idd_body_vel_x = idd_body_vel_x_array[steady_state_mask]
steady_state_idd_body_vel_y = idd_body_vel_y_array[steady_state_mask]
steady_state_idd_body_vel_yaw = idd_body_vel_yaw_array[steady_state_mask]

steady_state_icp_body_vel_x = icp_vel_x_array[steady_state_mask]
steady_state_icp_body_vel_y = icp_vel_y_array[steady_state_mask]
steady_state_icp_body_vel_yaw = icp_vel_yaw_array[steady_state_mask]

steady_state_body_vel_disturption_x = body_vel_disturption_x_array[steady_state_mask]
steady_state_body_vel_disturption_y = body_vel_disturption_y_array[steady_state_mask]
steady_state_body_vel_disturption_yaw = body_vel_disturption_yaw_array[steady_state_mask]




In [4]:
# import GP model


length_scales_x = [0.1]
length_scales_y = [0.1]
length_scales_yaw = [100]
noise_levels_x = 0.1
noise_levels_y = 0.1
noise_levels_yaw = 0.5

n_restarts_optimizer = 20

# robot = 'husky'
robot = 'marmotte'
# robot = 'warthog-track'
if robot=='husky':
    wheel_radius = 0.33 / 2
    baseline = 0.55
    dt=0.05
if robot=='marmotte':
    wheel_radius = 0.116
    baseline = 0.5927
    dt=0.05
if robot == 'warthog-track':
    wheel_radius = 0.3
    baseline = 1.1652
    dt = 0.05
kappa_param = 1

full_body_slip_gp = FullBodySlipGaussianProcess(1, 1, 3, length_scales_x, length_scales_y, length_scales_yaw,
                 noise_levels_x, noise_levels_y, noise_levels_yaw, n_restarts_optimizer, baseline, wheel_radius, dt, kappa_param)

In [5]:
# train params
# params_path = '../data/ral2023_dataset/husky/boreal_snow/trained_params/slip/gp/steady-state/'
# params_path = '../data/ral2023_dataset/husky/grand_salon_tile_inflated/trained_params/slip/gp/steady-state/'
# params_path = '../data/ral2023_dataset/husky/grand_salon_left-deflated/trained_params/slip/gp/steady-state/'
# params_path = '../data/ral2023_dataset/marmotte/boreal_snow/trained_params/slip/gp/steady-state/'
# params_path = '../data/ral2023_dataset/marmotte/ga_hard_snow_a/trained_params/slip/gp/steady-state/'
# params_path = '../data/ral2023_dataset/marmotte/ga_hard_snow_b/trained_params/slip/gp/steady-state/'
params_path = '../data/ral2023_dataset/marmotte/grand_salon_tile_b/slip_dataset_all.pkl'
# params_path = '../data/ral2023_dataset/warthog_tracks/boreal_mud/trained_params/slip/gp/steady-state/'
# params_path = '../data/ral2023_dataset/warthog_tracks/grand-axe_crusted-snow/trained_params/slip/gp/steady-state/'

full_body_slip_gp.train_params(steady_state_idd_body_vel_x, steady_state_idd_body_vel_yaw, 
                               steady_state_body_vel_disturption_x, steady_state_body_vel_disturption_y, steady_state_body_vel_disturption_yaw, True)

if not os.path.exists(params_path):
        os.makedirs(params_path)
full_body_slip_gp.save_params(params_path)
# full_body_slip_gp.save_params('../data/ral2023_dataset/warthog_tracks/boreal_mud/trained_params/slip/gp/steady-state/')
# full_body_slip_gp.save_params('../data/ral2023_dataset/warthog_tracks/grand-axe_crusted-snow/trained_params/slip/gp/steady-state/')


/home/dominic/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


gpr_x
0.39572334064007875
0.0648**2 * RBF(length_scale=0.0163) + WhiteKernel(noise_level=3.29e-05)
gpr_y
0.5254290599179613
0.0716**2 * RBF(length_scale=0.000315) + WhiteKernel(noise_level=1e-05)
gpr_yaw
0.9844701591553835
0.308**2 * RBF(length_scale=2.18) + WhiteKernel(noise_level=1e-05)


/home/dominic/.local/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


In [6]:
full_body_slip_gp.load_params('../data/ral2023_dataset/warthog_tracks/boreal_mud/trained_params/slip/gp/steady-state/')

In [7]:
def compute_single_step_error(meas_state, pred_state):
    pred_error = meas_state - pred_state
    pred_error[2] = wrap2pi(pred_error[2]) # wrap yaw angle error to pi
    pred_error_squared_scalar = pred_error.T @ pred_error
    return np.sqrt(pred_error_squared_scalar)

window_size = 40
n_test_windows = len(validate_dataset)

idd_body_vel_x_array = validate_dataset[idd_body_vel_x_str_list].to_numpy()
idd_body_vel_y_array = validate_dataset[idd_body_vel_y_str_list].to_numpy()
idd_body_vel_yaw_array = validate_dataset[idd_body_vel_yaw_str_list].to_numpy()
icp_x_array = validate_dataset[icp_x_str_list].to_numpy()
icp_y_array = validate_dataset[icp_y_str_list].to_numpy()
icp_yaw_array = validate_dataset[icp_yaw_str_list].to_numpy()
body_vel_disturption_x_array = validate_dataset[str_body_vel_disturption_x_list].to_numpy()
body_vel_disturption_y_array = validate_dataset[str_body_vel_disturption_y_list].to_numpy()
body_vel_disturption_yaw_array = validate_dataset[str_body_vel_disturption_yaw_list].to_numpy()

robot='W'
if robot=='H':
    wheel_radius = 0.33 / 2
    baseline = 0.55
    dt = 0.05
if robot=='M':
    wheel_radius = 0.116
    baseline = 0.5927
    dt=0.05
if robot == 'W':
    wheel_radius = 0.3
    baseline = 1.1652
    dt = 0.05

slip_input_dimensions = 2
a_param_init = 0
b_param_init = 0
param_variance_init = 999999999999999999999
variance_init = 1000000000
kappa_param = 1

ideal_diff_drive = Ideal_diff_drive(wheel_radius, baseline, dt)

gp_predictions = np.zeros((n_test_windows, window_size, 3))
idd_predictions = np.zeros((n_test_windows, window_size, 3))
cmd_input_horizon = np.zeros((2, window_size))
transitory_input_horizon = np.zeros((2, window_size))
cmd_body_vel_horizon = np.zeros((2, window_size))
transitory_body_vel_horizon = np.zeros((3, window_size))
gp_slip_body_vel_horizon = np.zeros((3, window_size))

gp_slip_horizon_prediction = np.zeros((3, window_size))
icp_horizon_meas = np.zeros((3, window_size))

cmd_idd_prediction_errors = np.zeros(n_test_windows)
transitory_idd_prediction_errors = np.zeros(n_test_windows)
gp_slip_prediction_errors = np.zeros(n_test_windows)
idd_body_vels_array = np.zeros((window_size, 3))
gp_init_state_covariance = np.eye(3) * 0.1

for j in range(0, n_test_windows):

    idd_body_vels_array[:, 0] = idd_body_vel_x_array[j, :]
    idd_body_vels_array[:, 1] = idd_body_vel_y_array[j, :]
    idd_body_vels_array[:, 2] = idd_body_vel_yaw_array[j, :]

    prediction_means, prediction_covariances = full_body_slip_gp.predict_horizon_from_body_idd_vels(idd_body_vels_array, 
                                                                                                    gp_slip_horizon_prediction[:, 0], gp_init_state_covariance)
    # print(prediction_means)
    
    icp_horizon_meas[0, :] = icp_x_array[j, :]
    icp_horizon_meas[1, :] = icp_y_array[j, :]
    icp_horizon_meas[2, :] = icp_yaw_array[j, :]
    gp_prediction_errors = icp_horizon_meas - prediction_means
    
    gp_horizon_prediction_error = 0

    for i in range(0, window_size):
        icp_2d_state = np.array([icp_x_array[j, i], icp_y_array[j, i], icp_yaw_array[j, i]])
        gp_horizon_prediction_error += compute_single_step_error(icp_2d_state, prediction_means[:, i])

    # gp_msre = np.diag(gp_prediction_errors @ gp_prediction_errors.T)
    # gp_slip_prediction_errors[j] = np.sqrt(np.sqrt((gp_msre.T @ gp_msre) / window_size))

    # print('gp : ', prediction_means[2, -1])
    # print('icp : ', icp_horizon_meas[2, -1])


    gp_slip_prediction_errors[j] = gp_horizon_prediction_error / window_size
